In [ ]:
import azureml.core
print("SDK version: ", azureml.core.VERSION)

In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2', max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

In [ ]:
from azureml.core import Experiment

experiment_name = 'pyodbc_exp'
experiment = Experiment(ws, name=experiment_name)

In [ ]:
from azureml.core import Environment
env = Environment(name='myenv')

In [ ]:
env.docker.enabled = True
dockerfile = None
with open('./Dockerfile', "r") as f:
    dockerfile = f.read()
env.docker.base_dockerfile = dockerfile

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies
conda_dep = CondaDependencies()
conda_dep.add_conda_package('pyodbc')
env.python.conda_dependencies = conda_dep

In [ ]:
from azureml.train.estimator import Estimator
est = Estimator(source_directory='.', compute_target=compute_target, entry_script='test.py', environment_definition=env)

In [ ]:
run = experiment.submit(est)
print(run)

In [ ]:
run.wait_for_completion(show_output=True)

In [ ]:
print("finished")